## Init DB

<div style="text-align: center;">
    <img src="DB-Schema.drawio.png" alt="DB-Schema.png" style="width: 500px;"/>
</div>

<div style="text-align: center;">
    <img src="Layout.png" alt="DB-Schema.png" style="width: 500px;"/>
</div>

In [1]:
import sqlite3


# neue Datenbank erstellen
conn = sqlite3.connect('prod_data.db')
cursor = conn.cursor()

# Produktstammdaten 
cursor.execute('''create table if not exists produktstammdaten (
        id varchar primary key unique,  
        bezeichnung text,
        dlz_plan integer,
        losgroesse integer,
        Bestand_RTL integer,
        Bestand_FTL integer,
        Bestellbestand integer)''') # weitere Spalten sind zu definieren

# Aufträge
cursor.execute('''create table if not exists auftraege (
        Charge integer primary key autoincrement,
        id varchar,
        stueckzahl integer,
        freigabe integer,
        fertigstellung integer, 
        FOREIGN KEY (id) REFERENCES produktstammdaten(id))''') # weitere Spalten sind zu definieren

# Betriebsmittel
cursor.execute('''create table if not exists betriebsmittel (
        bmg varchar primary key,
        bezeichnung varchar,
        knoten varchar)''') # weitere Spalten sind zu definieren

# Arbeitspläne
cursor.execute('''create table if not exists arbeitsplaene (
        id varchar,
        nr varchar,
        bmg varchar,
        r_plan integer,
        t_plan integer,
        PRIMARY KEY (id, nr),
        FOREIGN KEY (id) REFERENCES produktstammdaten(id))''') # weitere Spalten sind zu definieren

# Flurförderzeuge
cursor.execute('''create table if not exists ffz (
        id varchar primary key unique,
        speed integer,
        akkukapazitaet integer)''') # weitere Spalten sind zu definieren

# Fertigungs-Log-Files = FLF
cursor.execute('''create table if not exists FLF (
        Charge varchar,
        bmg varchar,
        ankunft integer,
        start_ruesten integer,
        start_bearbeitung integer,
        ende_bearbeitung integer,
        abtransport integer,
        primary key (Charge, bmg),
        FOREIGN KEY (bmg) REFERENCES betriebsmittel(bmg),
        FOREIGN KEY (charge) REFERENCES auftraege(charge))''') # weitere Spalten sind zu definieren

# Transport-Log-Files = TLF
cursor.execute('''create table if not exists TLF (
        Vorgangs_nr integer primary key autoincrement,
        FFZ_id varchar,
        start_knoten varchar,
        end_knoten varchar,
        startzeitpunkt integer,
        endzeitpunkt integer, 
        akkustand integer,       
        charge varchar,
        FOREIGN KEY (charge) REFERENCES auftraege(charge))''') # weitere Spalten sind zu definieren

# Setze den Startwert von Charge auf 20000000
cursor.execute('INSERT INTO sqlite_sequence (name, seq) VALUES ("auftraege", 19999999)')
# Setze den Startwert von Charge auf 1
cursor.execute('INSERT INTO sqlite_sequence (name, seq) VALUES ("TLF", 0)')

# Stammdaten pflegen
cursor.execute('INSERT INTO produktstammdaten (id, bezeichnung, dlz_plan, losgroesse, bestand_rtl, bestand_ftl, bestellbestand) VALUES (?, ?, ?, ?, ?, ?, ?)', 
               ('A2', 'Deckel', '150', '50', '1000', '1000', '1500'))
cursor.execute('INSERT INTO produktstammdaten (id, bezeichnung, dlz_plan, losgroesse, bestand_rtl, bestand_ftl, bestellbestand) VALUES (?, ?, ?, ?, ?, ?, ?)', 
               ('A7', 'Welle', '50', '200', '1000', '1000', '1700'))

# Arbeitspläne
cursor.execute('INSERT INTO arbeitsplaene (id, nr, bmg, r_plan, t_plan) VALUES (?, ?, ?, ?, ?)',
               ('A2', '10', 'SAE', '10', '20'))
cursor.execute('INSERT INTO arbeitsplaene (id, nr, bmg, r_plan, t_plan) VALUES (?, ?, ?, ?, ?)', 
               ('A2', '20', 'DRH', '15', '60'))
cursor.execute('INSERT INTO arbeitsplaene (id, nr, bmg, r_plan,t_plan) VALUES (?, ?, ?, ?, ?)', 
               ('A2', '30', 'QPR', '12', '30'))
cursor.execute('INSERT INTO arbeitsplaene (id, nr, bmg, r_plan, t_plan) VALUES (?, ?, ?, ?, ?)', 
               ('A7', '10', 'SAE', '8', '20'))
cursor.execute('INSERT INTO arbeitsplaene (id, nr, bmg, r_plan, t_plan) VALUES (?, ?, ?, ?, ?)', 
               ('A7', '20', 'DRH', '10', '70'))
cursor.execute('INSERT INTO arbeitsplaene (id, nr, bmg, r_plan, t_plan) VALUES (?, ?, ?, ?, ?)', 
               ('A7', '30', 'QPR', '5', '20'))

# Fertigungsplätze
cursor.executemany('insert into betriebsmittel (bmg, bezeichnung, knoten) values (?,?,?)',
                [
                ('RTL', 'Rohteillager', 'a'),
                ('SAE', 'Zuschnitt', 'b'),
                ('DRH1', 'Drehen Nr. 1', 'c'),
                ('DRH2', 'Drehen Nr. 2', 'c'),
                ('FRA', 'Fräsen', 'd'),
                ('QPR', 'Qualitätsprüfung', 'e'),
                ('FTL', 'Fertigteillager', 'f'),
                ])


# FFZ
cursor.execute('INSERT INTO ffz (id, speed, akkukapazitaet) VALUES (?, ?, ?)', ('F1', '10', '300'))
cursor.execute('INSERT INTO ffz (id, speed, akkukapazitaet) VALUES (?, ?, ?)', ('F2', '14', '450'))
cursor.execute('INSERT INTO ffz (id, speed, akkukapazitaet) VALUES (?, ?, ?)', ('F3', '12', '290'))


# Änderungen speichern
conn.commit()

# Verbindung schließen
conn.close()

## Check DB

In [2]:
# Verbindung zur bestehenden Datenbank herstellen
conn = sqlite3.connect('prod_data.db')
cursor = conn.cursor()

# Abfrage, um alle Tabellen in der Datenbank anzuzeigen
cursor.execute('''
    SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';
''')

# Alle Tabellen auflisten
tables = cursor.fetchall()

# Für jede Tabelle den Inhalt anzeigen
for table in tables:
    table_name = table[0]
    print(f"Inhalt der Tabelle: {table_name}")
    
    # Inhalt der Tabelle abfragen
    cursor.execute(f'SELECT * FROM {table_name}')
    rows = cursor.fetchall()
    
    # Spaltennamen der Tabelle anzeigen
    column_names = [description[0] for description in cursor.description]
    print(column_names)  # Spaltennamen drucken
    
    # Inhalt der Tabelle Zeile für Zeile drucken
    for row in rows:
        print(row)
    print("\n" + "-"*50 + "\n")  # Trennlinie für bessere Lesbarkeit

# Verbindung schließen
conn.close()

Inhalt der Tabelle: produktstammdaten
['id', 'bezeichnung', 'dlz_plan', 'losgroesse', 'Bestand_RTL', 'Bestand_FTL']
('A2', 'Deckel', 150, 50, 1000, 2000)
('A7', 'Welle', 50, 200, 1000, 2000)

--------------------------------------------------

Inhalt der Tabelle: auftraege
['Charge', 'id', 'stueckzahl']

--------------------------------------------------

Inhalt der Tabelle: betriebsmittel
['bmg', 'bezeichnung', 'knoten']
('RTL', 'Rohteillager', 'a')
('SAE', 'Zuschnitt', 'b')
('DRH1', 'Drehen Nr. 1', 'c')
('DRH2', 'Drehen Nr. 2', 'c')
('FRA', 'Fräsen', 'd')
('QPR', 'Qualitätsprüfung', 'e')
('FTL', 'Fertigteillager', 'f')

--------------------------------------------------

Inhalt der Tabelle: arbeitsplaene
['id', 'nr', 'bmg', 'r_plan', 't_plan']
('A2', '10', 'SAE', 10, 70)
('A2', '20', 'DRH', 15, 50)
('A2', '30', 'QPR', 12, 60)
('A7', '10', 'SAE', 8, 40)
('A7', '20', 'DRH', 10, 70)
('A7', '30', 'QPR', 5, 20)

--------------------------------------------------

Inhalt der Tabelle: ffz
[